# **Core: Subspace Alignment (SA)**

In [ ]:
import numpy as np

def subspace_alignment(source_data, target_data, subspace_dim):
  # Convert input data to numpy arrays
  source_data = np.array(source_data)
  target_data = np.array(target_data)

  # Define the normalize_data function
  def normalize_data(data):
    mean_data = np.mean(data, axis=0)
    std_data = np.std(data, axis=0)
    normalized_data = (data - mean_data) / std_data
    return normalized_data

  # Normalize source and target data
  source_data_norm = normalize_data(source_data)
  target_data_norm = normalize_data(target_data)

  # Perform PCA on source and target data
  def perform_pca(data):
    cov_matrix = np.cov(data.T)
    eig_val, eig_vec = np.linalg.eig(cov_matrix)
    idx = eig_val.argsort()[::-1]
    eig_val = eig_val[idx]
    eig_vec = eig_vec[:, idx]
    return eig_vec

  # Compute PCA for source and target data
  xs = perform_pca(source_data_norm)
  xt = perform_pca(target_data_norm)

  # Generating the subspaces
  subspace_dim = min(subspace_dim, min(xs.shape[1], xt.shape[1]))
  xs = xs[:, :subspace_dim]
  xt = xt[:, :subspace_dim]

  # Subspace alignment and project data
  target_aligned_source_data = source_data_norm @ (xs @ xs.T @ xt)
  target_projected_data = target_data_norm @ xt
  target_subspace = xt

  return target_aligned_source_data, target_projected_data, target_subspace

# **Main Code: Enhanced Subspace Alignment with Clustering and Weighting**
* Working on SEED dataset
* Target Subjects: 1-5
* **Important Note**: Parallel versions should be used for other subjects!

In [ ]:
import os
import sys
import time
import math
import random as python_random
import random as rn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import classification_report

import scipy.io

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.math import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# Set the random seeds
os.environ['PYTHONHASHSEED']= '123'
os.environ['TF_CUDNN_DETERMINISTIC']= '1'
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(123)
SEED_num = 40

sel_ch = np.arange(1,63,1)
num_ch = 62
for i in range(0,len(sel_ch)):
  sel_ch[i] = sel_ch[i] - 1

labels = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/label.mat')

# Paths for preprocessed data
path_sess_1  = ['1_20131027.mat','2_20140404.mat','3_20140603.mat','4_20140621.mat','5_20140411.mat',
                '6_20130712.mat','7_20131027.mat','8_20140511.mat','9_20140620.mat','10_20131130.mat',
                '11_20140618.mat','12_20131127.mat','13_20140527.mat','14_20140601.mat','15_20130709.mat']

path_sess_2  = ['1_20131030.mat','2_20140413.mat','3_20140611.mat','4_20140702.mat','5_20140418.mat',
                '6_20131016.mat','7_20131030.mat','8_20140514.mat','9_20140627.mat','10_20131204.mat',
                '11_20140625.mat','12_20131201.mat','13_20140603.mat','14_20140615.mat','15_20131016.mat']

path_sess_3  = ['1_20131107.mat','2_20140419.mat','3_20140629.mat','4_20140705.mat','5_20140506.mat',
                '6_20131113.mat','7_20131106.mat','8_20140521.mat','9_20140704.mat','10_20131211.mat',
                '11_20140630.mat','12_20131207.mat','13_20140610.mat','14_20140627.mat','15_20131105.mat']

# num_feature is a variable to select different features ----> DE feature: num_feature=4
num_feature = 4
label_sess_1 = labels['label']

# Select values ​​for parameters to execute the code
sub_dim = 40
Source_SESS_A = 1
Source_SESS_B = 3
Target_SESS = 2

print("Source_Session_A = "+str(Source_SESS_A) +"  +  Source_Session_B = "+str(Source_SESS_B)+"  >>>>>>>>>>>  Traget_Session = "+str(Target_SESS)+"\n\n")

PATH = [path_sess_1, path_sess_2, path_sess_3]
path_Source_SESS_A = PATH[Source_SESS_A-1]
path_Source_SESS_B = PATH[Source_SESS_B-1]
path_Target_SESS = PATH[Target_SESS-1]

#Loading and Preparing the Cross-Subject Accuracy Matrix According to the Selected Targer Session
cross_dir = '/content/drive/MyDrive/SubjectClustering_SEED_AllSessions/Cross_Acc_OneSessionTrainTrain'+str(Target_SESS)+'.csv'
ACC_MAT= np.genfromtxt(cross_dir, delimiter=',')
ACC_MAT = ACC_MAT.reshape(15,15,5)

ACC_MAT_iter = np.zeros((15,5))
Target_acc_list_selected_MMPP = []
Target_acc_list_selected_MNCC = []
Target_acc_list_selected_MMA = []
Target_acc_list_selected_MTA = []
Target_acc_list_clustering = []
Target_acc_list_HIGHEST = []

# Important Note: This version of the code is prepared for the first 5 subjects.
# Parallel versions should be used for other subjects!

for target_sub in range(1,6):

  ############ 1. Source Subjects Clustering ############
  source_sub = np.concatenate((np.arange(1,target_sub),np.arange(target_sub+1, 16)),0)
  print("\n\nTarget Subject: "+str(target_sub))
  print("Source Subjects: "+str(source_sub)+"\n")
  iter_acc_list_selected_MMPP = []
  iter_acc_list_selected_MNCC = []
  iter_acc_list_selected_MMA = []
  iter_acc_list_selected_MTA = []
  iter_acc_list_clustering = []
  iter_acc_list_HIGHEST = []

  for iter in range(0,5):
    all_trials = [*range(0,15)]
    valid_trials = [*range(3*iter,3*(iter+1))]
    train_trials = np.concatenate((np.arange(0,3*iter),np.arange((3*iter+3), 15)),0)
    print("\niteration "+str(iter+1)+" started!")
    print("\ntrian trials: " + str(train_trials))
    print("valid trials: " + str(valid_trials)+"\n")

    ACC_MAT_iteration = ACC_MAT[:,:,iter]
    Cross_Acc = np.delete(ACC_MAT_iteration,target_sub-1,0)
    Cross_Acc = np.delete(Cross_Acc,target_sub-1,1)

    SILHO =[]
    for num_cluster in range(2,5):
      Kmean = KMeans(n_clusters=num_cluster, random_state=0)
      cluster_labels = Kmean.fit_predict(Cross_Acc)
      silhouette_avg = silhouette_score(Cross_Acc, cluster_labels)
      SILHO.append(silhouette_avg)
    SILHO = np.array(SILHO)
    print("silhouette array:" + str(SILHO))
    best_num_cluster = np.argmax(SILHO) + 2
    print("the best number of clusters: " +str(best_num_cluster))

    Kmean = KMeans(n_clusters=best_num_cluster, random_state=0)
    cluster_labels = Kmean.fit_predict(Cross_Acc)
    print("cluster labels: "+str(cluster_labels))
    clusters_source_index =[]
    num_cluster_samples =[]
    for cluster in range(0,best_num_cluster):
      cluster_list = np.where(np.equal(cluster_labels, cluster))
      cluster_list = np.array(cluster_list)
      cluster_list = np.squeeze(cluster_list)
      if cluster_list.shape ==():
        temp = cluster_list
        x = temp.astype(int)
        cluster_list = np.array([x])
      list_sub = list(cluster_list)
      clusters_source_index.append(list_sub)
      num_cluster_samples.append(len(list_sub))
    clusters_index = np.array(clusters_source_index)
    cluster_num_of_sources = np.array(num_cluster_samples)
    print("number of subjects in clusters: " +str(num_cluster_samples))
    print("cluster_source_index :" +str(clusters_source_index))

    # Cluster Selection based on the Clustering Method
    columns = np.concatenate((np.arange(0,target_sub-1),np.arange((target_sub), 15)),0)
    data_target = ACC_MAT_iteration[target_sub-1,columns]
    data_target = data_target.reshape(1,-1)
    K_clustering = int(Kmean.predict(data_target))
    print("K (Selected Cluster): "+str(K_clustering+1))
    num_of_SubSess = num_cluster_samples[K_clustering]
    index_of_SubSess = clusters_source_index[K_clustering]

    Source_Subspace = np.zeros((14,num_ch,sub_dim))
    subj_row = 0
    init_A = 1   # for features matrix initialization
    init_B = 1   # for features matrix initialization
    init_Target = 1  # for features matrix initialization

    # Preparation the samples
    for p in source_sub:
      ######################### Sourcce Session A ##################################
      fetures_Source_SESS_A  = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/' + path_Source_SESS_A[p-1])
      feature_delta = []
      feature_tetha = []
      feature_alpha = []
      feature_beta  = []
      feature_gamma = []

      field_name = []
      for key in fetures_Source_SESS_A.keys():
          field_name.append(key)

      label = []
      for i in all_trials:
        data_trial = fetures_Source_SESS_A[field_name[12*i+num_feature]]
        for k in range(140,data_trial.shape[1]):
          data_epoch   = data_trial[:,k,:]
          data_epoch   = np.squeeze(data_epoch)
          a = []
          b = []
          c = []
          d = []
          e = []
          for j in range(0,num_ch):
            data_channel = data_epoch[sel_ch[j],:]
            a.append(data_channel[0])
            b.append(data_channel[1])
            c.append(data_channel[2])
            d.append(data_channel[3])
            e.append(data_channel[4])

          feature_delta.append(np.array(a))
          feature_tetha.append(np.array(b))
          feature_alpha.append(np.array(c))
          feature_beta.append(np.array(d))
          feature_gamma.append(np.array(e))
          label.append(label_sess_1[0,i])

      feature_delta = np.array(feature_delta)
      feature_tetha = np.array(feature_tetha)
      feature_alpha = np.array(feature_alpha)
      feature_beta  = np.array(feature_beta)
      feature_gamma = np.array(feature_gamma)
      label        = np.array(label)
      all_features = feature_gamma

      num_sample = all_features.shape[0]
      if init_A==1:
        raw_sources_features_A = np.zeros((14,num_sample,num_ch))
        raw_sources_label_A = np.zeros((14,num_sample))
        init_A = 0

      ## label and data for Source Session A set
      data_train_S_A  = all_features
      label_train_S_A = label

      raw_sources_features_A[subj_row,:,:] = data_train_S_A
      raw_sources_label_A[subj_row,:] = label_train_S_A

      ######################### Source Session B ##################################
      fetures_Source_SESS_B  = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/' + path_Source_SESS_B[p-1])
      feature_delta = []
      feature_tetha = []
      feature_alpha = []
      feature_beta  = []
      feature_gamma = []

      field_name = []
      for key in fetures_Source_SESS_B.keys():
          field_name.append(key)

      label = []
      for i in all_trials:
        data_trial = fetures_Source_SESS_B[field_name[12*i+num_feature]]
        for k in range(140,data_trial.shape[1]):
          data_epoch   = data_trial[:,k,:]
          data_epoch   = np.squeeze(data_epoch)
          a = []
          b = []
          c = []
          d = []
          e = []
          for j in range(0,num_ch):
            data_channel = data_epoch[sel_ch[j],:]
            a.append(data_channel[0])
            b.append(data_channel[1])
            c.append(data_channel[2])
            d.append(data_channel[3])
            e.append(data_channel[4])

          feature_delta.append(np.array(a))
          feature_tetha.append(np.array(b))
          feature_alpha.append(np.array(c))
          feature_beta.append(np.array(d))
          feature_gamma.append(np.array(e))
          label.append(label_sess_1[0,i])

      feature_delta = np.array(feature_delta)
      feature_tetha = np.array(feature_tetha)
      feature_alpha = np.array(feature_alpha)
      feature_beta  = np.array(feature_beta)
      feature_gamma = np.array(feature_gamma)
      label        = np.array(label)
      all_features = feature_gamma

      num_sample = all_features.shape[0]
      if init_B==1:
        raw_sources_features_B = np.zeros((14,num_sample,num_ch))
        raw_sources_label_B = np.zeros((14,num_sample))
        init_B = 0

      ## label and data for Source Session B set
      data_train_S_B  = all_features
      label_train_S_B = label

      raw_sources_features_B[subj_row,:,:] = data_train_S_B
      raw_sources_label_B[subj_row,:] = label_train_S_B

      ######################### Target Session ##################################
      fetures_Source_SESS_Target  = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/' + path_Target_SESS[p-1])
      feature_delta = []
      feature_tetha = []
      feature_alpha = []
      feature_beta  = []
      feature_gamma = []

      field_name = []
      for key in fetures_Source_SESS_Target.keys():
          field_name.append(key)

      label = []
      for i in all_trials:
        data_trial = fetures_Source_SESS_Target[field_name[12*i+num_feature]]
        for k in range(140,data_trial.shape[1]):
          data_epoch   = data_trial[:,k,:]
          data_epoch   = np.squeeze(data_epoch)
          a = []
          b = []
          c = []
          d = []
          e = []
          for j in range(0,num_ch):
            data_channel = data_epoch[sel_ch[j],:]
            a.append(data_channel[0])
            b.append(data_channel[1])
            c.append(data_channel[2])
            d.append(data_channel[3])
            e.append(data_channel[4])

          feature_delta.append(np.array(a))
          feature_tetha.append(np.array(b))
          feature_alpha.append(np.array(c))
          feature_beta.append(np.array(d))
          feature_gamma.append(np.array(e))
          label.append(label_sess_1[0,i])

      feature_delta = np.array(feature_delta)
      feature_tetha = np.array(feature_tetha)
      feature_alpha = np.array(feature_alpha)
      feature_beta  = np.array(feature_beta)
      feature_gamma = np.array(feature_gamma)
      label        = np.array(label)
      all_features = feature_gamma

      num_sample = all_features.shape[0]
      if init_Target==1:
        raw_sources_features_Target = np.zeros((14,num_sample,num_ch))
        raw_sources_label_Target = np.zeros((14,num_sample))
        init_Target = 0

      ## label and data for Target Session
      data_train_S_Target  = all_features
      label_train_S_Target = label

      raw_sources_features_Target[subj_row,:,:] = data_train_S_Target
      raw_sources_label_Target[subj_row,:] = label_train_S_Target
      subj_row += 1

    # Augmented sources data: align each session of each subject with the classifier's corresponding target session
    Aggregated_Sessions_shape = 3*raw_sources_features_A.shape[1]
    Augmented_sources_features = np.zeros((14,14,Aggregated_Sessions_shape,sub_dim))
    TargetSess_Projected_Data_AllSources = np.zeros((14,raw_sources_features_Target.shape[1],sub_dim))
    Augmented_Sources_label = np.zeros((14,14,Aggregated_Sessions_shape))
    for source in range(14):
      Source_Data_Sess_A = raw_sources_features_A[source,:,:]
      Source_Data_Sess_B = raw_sources_features_B[source,:,:]
      Source_Data_Sess_Target = raw_sources_features_Target[source,:,:]
      TargetSession_Aligned_Source_Data_A, Target_Projected_Data, _ = subspace_alignment(Source_Data_Sess_A, Source_Data_Sess_Target, sub_dim)
      TargetSession_Aligned_Source_Data_B, Target_Projected_Data, _ = subspace_alignment(Source_Data_Sess_B, Source_Data_Sess_Target, sub_dim)
      TargetSess_Projected_Data_AllSources[source,:,:] = Target_Projected_Data
      Augmented_sources_features[source,source,:,:] = np.concatenate((TargetSession_Aligned_Source_Data_A, TargetSession_Aligned_Source_Data_B, Target_Projected_Data),0)
      Augmented_Sources_label[source,source,:] = np.concatenate((raw_sources_label_A[source,:], raw_sources_label_B[source,:], raw_sources_label_Target[source,:]),0)
      other_sources = [*range(0,14)]
      other_sources.remove(source)
      for other_source in other_sources:
        Other_Source_Data_Sess_A = raw_sources_features_A[other_source,:,:]
        Other_Source_Data_Sess_B = raw_sources_features_B[other_source,:,:]
        Other_Source_Data_Sess_Target = raw_sources_features_Target[other_source,:,:]
        TargetSession_Aligned_Other_Source_Data_A, _, _ = subspace_alignment(Other_Source_Data_Sess_A, Source_Data_Sess_Target, sub_dim)
        TargetSession_Aligned_Other_Source_Data_B, _, _ = subspace_alignment(Other_Source_Data_Sess_B, Source_Data_Sess_Target, sub_dim)
        TargetSession_Aligned_Other_Source_Data_Target, _, _ = subspace_alignment(Other_Source_Data_Sess_Target, Source_Data_Sess_Target, sub_dim)
        Augmented_sources_features[source,other_source,:,:] = np.concatenate((TargetSession_Aligned_Other_Source_Data_A, TargetSession_Aligned_Other_Source_Data_B, TargetSession_Aligned_Other_Source_Data_Target),0)
        Augmented_Sources_label[source,other_source,:] = np.concatenate((raw_sources_label_A[other_source,:], raw_sources_label_B[other_source,:], raw_sources_label_Target[other_source,:]),0)


    ############ 2.Training the Classifiers ############
    # we will train have 14 classifiers here (5 classifiers in this version of the code)
    # train a 4-layer MLP for each source subject based on the corresponding augmented source data
    # we save the classifiers for next steps which use them for optimal cluster selection step

    print("\nTraining the Classifiers Step:\n")

    for source in range (1,15):
      # data preparing
      clust = cluster_labels[source-1]
      same_cluster = clusters_index[clust]
      print("tarining the source case: "+str(source-1)+"\n")
      #print("all sources in this cluster: " +str(same_cluster))
      x_train = Augmented_sources_features[source-1,:,:,:]
      x_train = x_train[same_cluster,:,:]

      # weighting the Target Session of each classifier corresponding source
      if cluster_num_of_sources[clust]!=1:
        weight = (3*((cluster_num_of_sources[clust])-1))-1
      else:
        weight = 1
      #print("weight: "+str(weight))
      source_TargetSess_weighted_data = np.repeat(TargetSess_Projected_Data_AllSources[source-1,:,:],weight,axis=0)
      source_TargetSess_weighted_labels = np.repeat(raw_sources_label_Target[source-1,:],weight)

      #print("x_train.shape before reshape: "+str(x_train.shape))
      x_train = x_train.reshape((cluster_num_of_sources[clust])*Aggregated_Sessions_shape,sub_dim)
      #print("x_train.shape after reshape: "+str(x_train.shape))

      x_train = np.concatenate((x_train, source_TargetSess_weighted_data),0)

      #print("x_train.shape after concatenating with weighted source data: "+str(x_train.shape))

      # label preparing
      y_train = Augmented_Sources_label[source-1,:,:]
      y_train = y_train[same_cluster,:]
      y_train = y_train.reshape(1,-1)
      y_train = np.squeeze(y_train)
      #print("y_train.shape before concatenating with weighted source labels: "+str(y_train.shape))

      y_train = np.concatenate((y_train, source_TargetSess_weighted_labels),0)
      #print("y_train.shape after concatenating with weighted source labels: "+str(y_train.shape))

      # Shuffle the Dataset and One-Hot the labels
      x_train , y_train = shuffle(x_train, y_train,random_state=SEED_num)
      y_train = tf.keras.utils.to_categorical(np.array(y_train),3)

      initializer = tf.keras.initializers.GlorotNormal(seed=24)

      model = Sequential([
          Flatten(input_shape=(sub_dim,)),
          Dense(32, activation= 'tanh', kernel_initializer=initializer, name='Dense_1'),
          Dense(20, activation='tanh', kernel_initializer=initializer, name='Dense_2'),
          Dense(10, activation='tanh', kernel_initializer=initializer, name='Dense_3'),
          Dense(3, activation='softmax', kernel_initializer=initializer, name='output')
      ])

      model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
      tf.random.set_seed(123)
      history = model.fit(x_train, y_train, epochs = 20, batch_size= 256 ,shuffle = False, verbose=0)

      model.save('/content/drive/MyDrive/SubjectClustering_SEED_AllSessions_MyVersion/classifiers/'+str(target_sub)+'-'+ str(source) +'-'+str(iter+1))

      # plotting the accuracy and loss curves (if needed, otherwise SKIP them)

      #plt.plot(history.history['loss'])
      #plt.plot(history.history['val_loss'])
      #plt.title("Loss Vs Epochs [case: "+str(source)+"][iteration: "+str(iter+1)+"]")
      #plt.ylabel('Loss')
      #plt.xlabel('Epochs')
      #plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
      #plt.show()

      #plt.plot(history.history['accuracy'])
      #plt.plot(history.history['val_accuracy'])
      #plt.title("Accuracy Vs Epochs [case: "+str(source)+"][iteration: "+str(iter+1)+"]")
      #plt.ylabel('Accuracy')
      #plt.xlabel('Epochs')
      #plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
      #plt.show()

    ############ 3. Cluster Selection ############


    ################# Target Training Data #################

    ########## Target training data trials (Session A) ##########
    features_Target_A = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/' + path_Source_SESS_A[target_sub-1])

    feature_delta = []
    feature_tetha = []
    feature_alpha = []
    feature_beta  = []
    feature_gamma = []

    field_name = []
    for key in features_Target_A.keys():
        field_name.append(key)

    label = []
    for i in all_trials:
      data_trial = features_Target_A[field_name[12*i+num_feature]]
      for k in range(140,data_trial.shape[1]):
        data_epoch   = data_trial[:,k,:]
        data_epoch   = np.squeeze(data_epoch)
        a = []
        b = []
        c = []
        d = []
        e = []
        for j in range(0,num_ch):
          data_channel = data_epoch[sel_ch[j],:]
          a.append(data_channel[0])
          b.append(data_channel[1])
          c.append(data_channel[2])
          d.append(data_channel[3])
          e.append(data_channel[4])
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        d = np.array(d)
        e = np.array(e)

        feature_delta.append(a)
        feature_tetha.append(b)
        feature_alpha.append(c)
        feature_beta.append(d)
        feature_gamma.append(e)
        label.append(label_sess_1[0,i])
    feature_delta = np.array(feature_delta)
    feature_tetha = np.array(feature_tetha)
    feature_alpha = np.array(feature_alpha)
    feature_beta  = np.array(feature_beta)
    feature_gamma = np.array(feature_gamma)
    label        = np.array(label)
    all_features = feature_gamma

    ## label and data for Target Subject's Source Session A set
    data_train_T_Sess_A  = all_features
    label_train_T_Sess_A = label

    ########## Target training data trials (Session B) ##########
    features_Target_Sess_B = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/' + path_Source_SESS_B[target_sub-1])

    feature_delta = []
    feature_tetha = []
    feature_alpha = []
    feature_beta  = []
    feature_gamma = []

    field_name = []
    for key in features_Target_Sess_B.keys():
        field_name.append(key)

    label = []
    for i in all_trials:
      data_trial = features_Target_Sess_B[field_name[12*i+num_feature]]
      for k in range(140,data_trial.shape[1]):
        data_epoch   = data_trial[:,k,:]
        data_epoch   = np.squeeze(data_epoch)
        a = []
        b = []
        c = []
        d = []
        e = []
        for j in range(0,num_ch):
          data_channel = data_epoch[sel_ch[j],:]
          a.append(data_channel[0])
          b.append(data_channel[1])
          c.append(data_channel[2])
          d.append(data_channel[3])
          e.append(data_channel[4])
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        d = np.array(d)
        e = np.array(e)

        feature_delta.append(a)
        feature_tetha.append(b)
        feature_alpha.append(c)
        feature_beta.append(d)
        feature_gamma.append(e)
        label.append(label_sess_1[0,i])
    feature_delta = np.array(feature_delta)
    feature_tetha = np.array(feature_tetha)
    feature_alpha = np.array(feature_alpha)
    feature_beta  = np.array(feature_beta)
    feature_gamma = np.array(feature_gamma)
    label        = np.array(label)
    all_features = feature_gamma

    ## label and data for Target Subject's Source Session B set
    data_train_T_Sess_B  = all_features
    label_train_T_Sess_B = label


    ########## Target training data trials (Target Session) ##########
    features_Test = scipy.io.loadmat('/content/drive/MyDrive/SEED/ExtractedFeatures/' + path_Target_SESS[target_sub-1])
    feature_delta = []
    feature_tetha = []
    feature_alpha = []
    feature_beta  = []
    feature_gamma = []

    field_name = []
    for key in features_Test.keys():
        field_name.append(key)

    label = []
    for i in train_trials:
      data_trial = features_Test[field_name[12*i+num_feature]]
      for k in range(140,data_trial.shape[1]):
        data_epoch   = data_trial[:,k,:]
        data_epoch   = np.squeeze(data_epoch)
        a = []
        b = []
        c = []
        d = []
        e = []
        for j in range(0,num_ch):
          data_channel = data_epoch[sel_ch[j],:]
          a.append(data_channel[0])
          b.append(data_channel[1])
          c.append(data_channel[2])
          d.append(data_channel[3])
          e.append(data_channel[4])
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        d = np.array(d)
        e = np.array(e)

        feature_delta.append(a)
        feature_tetha.append(b)
        feature_alpha.append(c)
        feature_beta.append(d)
        feature_gamma.append(e)
        label.append(label_sess_1[0,i])
    feature_delta = np.array(feature_delta)
    feature_tetha = np.array(feature_tetha)
    feature_alpha = np.array(feature_alpha)
    feature_beta  = np.array(feature_beta)
    feature_gamma = np.array(feature_gamma)
    label        = np.array(label)
    all_features = feature_gamma

    ## label and data for Target Subject's Target Session Set
    data_train_T  = all_features
    label_train_T = label

    ################# Target test data trials (from Target Session) #################
    feature_delta = []
    feature_tetha = []
    feature_alpha = []
    feature_beta  = []
    feature_gamma = []

    label = []
    for i in valid_trials:
      data_trial = features_Test[field_name[12*i+num_feature]]
      for k in range(140,data_trial.shape[1]):
        data_epoch   = data_trial[:,k,:]
        data_epoch   = np.squeeze(data_epoch)
        a = []
        b = []
        c = []
        d = []
        e = []
        for j in range(0,num_ch):
          data_channel = data_epoch[sel_ch[j],:]
          a.append(data_channel[0])
          b.append(data_channel[1])
          c.append(data_channel[2])
          d.append(data_channel[3])
          e.append(data_channel[4])
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        d = np.array(d)
        e = np.array(e)

        feature_delta.append(a)
        feature_tetha.append(b)
        feature_alpha.append(c)
        feature_beta.append(d)
        feature_gamma.append(e)
        label.append(label_sess_1[0,i])
    feature_delta = np.array(feature_delta)
    feature_tetha = np.array(feature_tetha)
    feature_alpha = np.array(feature_alpha)
    feature_beta  = np.array(feature_beta)
    feature_gamma = np.array(feature_gamma)
    label        = np.array(label)
    all_features = feature_gamma

    ## test labels and data from target subject's Target Session
    data_valid  = all_features
    label_valid = label

    # Projected Target (training) Data
    num_sample_T = data_train_T.shape[0]
    Projected_Target_Data = np.zeros((14,num_sample_T,sub_dim))
    for source_row in range(0,14):
      Source_Data = raw_sources_features_Target[source_row,:,:]
      Target_Data = data_train_T
      Projected_Target_Data[source_row,:,:], _, _ = subspace_alignment(Target_Data, Source_Data, sub_dim)

    # Projected Target (test) Data
    num_sample_test = data_valid.shape[0]
    Projected_Target_Data_test = np.zeros((14,num_sample_test,sub_dim))
    for source_row in range(0,14):
      Source_Data = raw_sources_features_Target[source_row,:,:]
      Target_Data = data_valid
      Projected_Target_Data_test[source_row,:,:], _, _ = subspace_alignment(Target_Data, Source_Data, sub_dim)

    # prediction matrix (for train)
    Prediction_Matrix = np.zeros((14,num_sample_T))
    Prediction_Probs = np.zeros((14,num_sample_T,3))
    for classifier in range(1,15):
      # load model
      model = tf.keras.models.load_model('/content/drive/MyDrive/SubjectClustering_SEED_AllSessions_MyVersion/classifiers/'+str(target_sub)+'-'+ str(classifier) +'-'+str(iter+1))

      # predict label for the samples
      input_samples = Projected_Target_Data[classifier-1,:,:]
      labels = model.predict(input_samples)
      Prediction_Matrix[classifier-1,:] = np.argmax(np.squeeze(model.predict(input_samples)),axis=1)
      Prediction_Probs[classifier-1,:,:] = np.squeeze(model.predict(input_samples))

      prediction_matrix_non_onehot = np.zeros((14,num_sample_T))
      for source_row in range(14):
        for sample in range(num_sample_T):
          if Prediction_Matrix[source_row,sample]==0:
            prediction_matrix_non_onehot[source_row,sample]=0
          elif Prediction_Matrix[source_row,sample]==1:
            prediction_matrix_non_onehot[source_row,sample]=1
          elif Prediction_Matrix[source_row,sample]==2:
            prediction_matrix_non_onehot[source_row,sample]=-1

    # Prediction matrix (for test)
    Prediction_Matrix_test = np.zeros((14,num_sample_test))
    Prediction_Probs_test = np.zeros((14,num_sample_test,3))
    for classifier in range(1,15):
      # load model
      model = tf.keras.models.load_model('/content/drive/MyDrive/SubjectClustering_SEED_AllSessions_MyVersion/classifiers/'+str(target_sub)+'-'+ str(classifier) +'-'+str(iter+1))

      # predict label for the samples
      input_samples = Projected_Target_Data_test[classifier-1,:,:]
      labels = model.predict(input_samples)
      Prediction_Matrix_test[classifier-1,:] = np.argmax(np.squeeze(model.predict(input_samples)),axis=1)
      Prediction_Probs_test[classifier-1,:,:] = np.squeeze(model.predict(input_samples))

    prediction_matrix_test_non_onehot = np.zeros((14,num_sample_test))
    for source_row in range(14):
      for sample in range(num_sample_test):
        if Prediction_Matrix_test[source_row,sample]==0:
          prediction_matrix_test_non_onehot[source_row,sample]=0
        elif Prediction_Matrix_test[source_row,sample]==1:
          prediction_matrix_test_non_onehot[source_row,sample]=1
        elif Prediction_Matrix_test[source_row,sample]==2:
          prediction_matrix_test_non_onehot[source_row,sample]=-1

    # ********* Supervised Optimal Cluster Selection Metrics *********

    #MMPP: Maximum Major Prediction Proportion
    # Optimal cluster selection (based on unlabled test trials)
    clusters_proportions = np.zeros((best_num_cluster))
    for cluster in range(0,best_num_cluster):
      proportion_array = np.zeros((num_sample_test))
      for sample in range(0,num_sample_test):
        sum_prob_0 = 0
        sum_prob_1 = 0
        sum_prob_min1 = 0
        counter_0 = 0
        counter_1 = 0
        counter_min1 = 0
        use_prob = 0
        for classifier in range(cluster_num_of_sources[cluster]):
          source_index = clusters_source_index[cluster][classifier]
          if prediction_matrix_test_non_onehot[source_index,sample]==0:
            counter_0 += 1
          elif prediction_matrix_test_non_onehot[source_index,sample]==1:
            counter_1 += 1
          elif prediction_matrix_test_non_onehot[source_index,sample]==-1:
            counter_min1 += 1
        if (((counter_0 == counter_1) and (counter_min1<counter_0)) or ((counter_0 == counter_min1)and(counter_1<counter_0)) or ((counter_1 == counter_min1)and(counter_0<counter_1)) or (counter_0 == counter_1 and counter_0 == counter_min1)):
          for classifier in range(cluster_num_of_sources[cluster]):
            sum_prob_0 += Prediction_Probs_test[source_index,sample,0]
            sum_prob_1 += Prediction_Probs_test[source_index,sample,1]
            sum_prob_min1 += Prediction_Probs_test[source_index,sample,2]
          use_prob = 1
        if use_prob==0:
          major_pred = max(counter_0,counter_1,counter_min1)
          proportion = major_pred/(cluster_num_of_sources[cluster])
        else:
          sum_array = np.array([sum_prob_0,sum_prob_1,sum_prob_min1])
          counter_array = np.array([counter_0,counter_1,counter_min1])
          major_pred = np.argmax(sum_array)
          proportion = counter_array[major_pred]/(cluster_num_of_sources[cluster])
        proportion_array[sample] = proportion
      Avg_proportion = np.mean(proportion_array)
      clusters_proportions[cluster] = Avg_proportion
    print("\n\nclusters proportions: "+str(clusters_proportions))
    K_Selected_MMPP = np.argmax(clusters_proportions)
    print('K (Selected Cluster)(MMPP): '+str(K_Selected_MMPP +1))

    #MCPCP: Maximum Correctly Predicted Classifiers Proportion
    # Optimal cluster selection (based on available Target training samples)
    clusters_proportions = np.zeros((best_num_cluster))
    for cluster in range(0,best_num_cluster):
      proportion_array = np.zeros((num_sample_T))
      for sample in range(0,num_sample_T):
        counter_correct = 0
        for classifier in range(cluster_num_of_sources[cluster]):
          source_index = clusters_source_index[cluster][classifier]
          if prediction_matrix_non_onehot[source_index,sample]==label_train_T[sample]:
            counter_correct += 1
        proportion_array[sample] = counter_correct/cluster_num_of_sources[cluster]
      clusters_proportions[cluster] = np.mean(proportion_array)
    print("\n\nclusters proportions: "+str(clusters_proportions))
    K_Selected_MNCC = np.argmax(clusters_proportions)
    print('K (Selected Cluster)(MNCC): '+str(K_Selected_MNCC +1))

    #Maximum Mean Accuracy (MMA) and Maximum Top Accuracy (MTA)
    # Optimal cluster selection (based on available Target training samples)
    mean_clusters_acc = np.zeros((best_num_cluster))
    high_clusters_acc = np.zeros((best_num_cluster))
    for cluster in range(best_num_cluster):
      classifiers_acc = np.zeros((cluster_num_of_sources[cluster]))
      for classifier in range(cluster_num_of_sources[cluster]):
        source_index = clusters_source_index[cluster][classifier]
        acc = (sum(np.equal(label_train_T,prediction_matrix_non_onehot[source_index,:][0:len(label_train_T)])))/len(label_train_T)
        classifiers_acc[classifier] = acc
      mean_clusters_acc[cluster] = np.mean(classifiers_acc)
      high_clusters_acc[cluster] = np.max(classifiers_acc)
    print("\n\nmean accuracy for classifiers in each cluster: "+str(mean_clusters_acc))
    print("high accuracy for classifiers in each cluster: "+str(high_clusters_acc))
    K_Selected_MMA = np.argmax(mean_clusters_acc)
    K_Selected_MTA = np.argmax(high_clusters_acc)
    print('K (MMA): '+str(K_Selected_MMA+1))
    print('K (MTA): '+str(K_Selected_MTA+1))
    print("\n\nK (Clustering): "+str(K_clustering+1)+"\n")

    ############ 4. Source Selection (Optional Step) ############

    final_accuracy =[]
    High_acc_in_iter = 0
    for k in range(best_num_cluster):
    #for k in [K_Selected,K_clustering]:
    #for k in [K_Selected]:
      K=k
      #################  train Booster Classifier on Target Subject's All Sessions Data   #################
      Booster_Data_Shape = 3*data_train_T_Sess_A.shape[0]
      Booster_TargetSess_Alinged_Sess_A, Booster_TargetSess_Projected,_ = subspace_alignment(data_train_T_Sess_A, data_train_T, sub_dim)
      Booster_TragetSess_Aligned_Sess_B,_,_ = subspace_alignment(data_train_T_Sess_B, data_train_T, sub_dim)
      Booster_Data = np.concatenate((Booster_TargetSess_Alinged_Sess_A, Booster_TragetSess_Aligned_Sess_B, Booster_TargetSess_Projected),0)
      Booster_label = np.concatenate((label_train_T_Sess_A, label_train_T_Sess_B, label_train_T),0)

      Booster_weight = (3*(cluster_num_of_sources[K]))-1
      Booster_TargetSess_Weighted_Data = np.repeat(Booster_TargetSess_Projected,Booster_weight,axis=0)
      Booster_TargetSess_Weighted_label = np.repeat(label_train_T,Booster_weight)

      other_sources_aggregated_shape = 3*raw_sources_features_A.shape[1]
      Booster_Selected_cluster_data = np.zeros((cluster_num_of_sources[K],other_sources_aggregated_shape,sub_dim))
      Booster_Selected_cluster_label = np.zeros((cluster_num_of_sources[K],other_sources_aggregated_shape))
      for source in range(cluster_num_of_sources[K]):
        source_index = clusters_source_index[K][source]
        Other_Source_Data_Sess_A = raw_sources_features_A[source_index,:,:]
        Other_Source_Data_Sess_B = raw_sources_features_B[source_index,:,:]
        Other_Source_Data_Sess_Target = raw_sources_features_Target[source_index,:,:]
        Booster_TargetSess_Aligned_Other_Source_Data_A, _, _ = subspace_alignment(Other_Source_Data_Sess_A, data_train_T, sub_dim)
        Booster_TargetSess_Aligned_Other_Source_Data_B, _, _ = subspace_alignment(Other_Source_Data_Sess_B, data_train_T, sub_dim)
        Booster_TargetSess_Aligned_Other_Source_Data_TargetSess, _, _ = subspace_alignment(Other_Source_Data_Sess_Target, data_train_T, sub_dim)
        Booster_TargetSess_Aligned_other_Source_Data = np.concatenate((Booster_TargetSess_Aligned_Other_Source_Data_A, Booster_TargetSess_Aligned_Other_Source_Data_B, Booster_TargetSess_Aligned_Other_Source_Data_TargetSess),0)
        Booster_TargetSess_Aligned_other_Source_label = np.concatenate((raw_sources_label_A[source_index,:], raw_sources_label_B[source_index,:], raw_sources_label_Target[source_index,:]),0)
        Booster_Selected_cluster_data[source,:,:] = Booster_TargetSess_Aligned_other_Source_Data
        Booster_Selected_cluster_label[source,:] = Booster_TargetSess_Aligned_other_Source_label
      Booster_Selected_cluster_data = Booster_Selected_cluster_data.reshape(cluster_num_of_sources[K]*other_sources_aggregated_shape,sub_dim)
      Booster_Selected_cluster_label = Booster_Selected_cluster_label.reshape(-1,1)
      Booster_Selected_cluster_label = np.squeeze(Booster_Selected_cluster_label)

      Booster_x_train = np.concatenate((Booster_Data,Booster_TargetSess_Weighted_Data,Booster_Selected_cluster_data),0)
      Booster_y_train = np.concatenate((Booster_label,Booster_TargetSess_Weighted_label,Booster_Selected_cluster_label),0)

      Booster_x_train , Booster_y_train = shuffle(Booster_x_train, Booster_y_train,random_state=SEED_num)
      Booster_y_train = tf.keras.utils.to_categorical(np.array(Booster_y_train),3)
      model_booster = Sequential([
            Flatten(input_shape=(sub_dim,)),
            Dense(32, activation= 'tanh', kernel_initializer=initializer, name='Dense_1'),
            Dense(20, activation='tanh', kernel_initializer=initializer, name='Dense_2'),
            Dense(10, activation='tanh', kernel_initializer=initializer, name='Dense_3'),
            Dense(3, activation='softmax', kernel_initializer=initializer, name='output')
        ])
      print("\ntarining the Booster Classifier for this cluster!\n")
      model_booster.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
      tf.random.set_seed(123)
      history = model_booster.fit(Booster_x_train, Booster_y_train, epochs = 5, batch_size= 256, shuffle = False, verbose=1)
      model_booster.save('/content/drive/MyDrive/SubjectClustering_SEED_AllSessions_MyVersion/classifiers/'+str(target_sub)+'-Booster-'+str(iter+1))

      CLUSTER_ACCURACY = np.zeros((cluster_num_of_sources[K]+1))
      cluster_k_accuracy = np.zeros((cluster_num_of_sources[K]))
      for classifier in range(cluster_num_of_sources[K]):
        classifier_index = clusters_source_index[K][classifier]
        pred_labels = prediction_matrix_non_onehot[classifier_index,:]
        ground_truth = label_train_T
        acc = (sum(np.equal(ground_truth,pred_labels[0:len(ground_truth)])))/len(ground_truth)
        cluster_k_accuracy[classifier] = acc
      #print("classifiers accuracy for selected cluster: "+str(cluster_k_accuracy))

      # Selecting top N classifiers with high accuracies
      #N = cluster_num_of_sources[K] if (cluster_num_of_sources[K]%2==1) else cluster_num_of_sources[K]-1
      #N = cluster_num_of_sources[K]
      #N = 3
      N_counter = 0
      for N in range(0,cluster_num_of_sources[K]+1):
      #for N in [0]:
        if N>0:
          sorted_indices = np.argsort(cluster_k_accuracy)
          idx = sorted_indices[::-1][:N]
          sorted_classifiers = []
          for i in range(N):
            sorted_classifiers.append(clusters_source_index[K][idx[i]])
          #print("selected source indices: "+str(sorted_classifiers))

          #align Target (test samples) with selected sources
          Aligned_Target = np.zeros((N,data_valid.shape[0],sub_dim))
          for source in range(N):
            source_index = sorted_classifiers[source]
            Selected_Source_Data = raw_sources_features_Target[source_index,:,:]
            Target_Data = data_valid
            Aligned_Target[source,:,:], _, _ = subspace_alignment(Target_Data, Selected_Source_Data , sub_dim)

        # align Traget (test samples) with Target training sample (Booster Classifier)
        Aligned_Target_Booster,_,_ = subspace_alignment(data_valid, data_train_T , sub_dim)

        # load models
        preds_matrix= np.zeros((N+1,data_valid.shape[0],3))
        selected_sources_accuracy = np.zeros((N+1))
        prediction = np.zeros(data_valid.shape[0])

        # Booster
        Booster_model = tf.keras.models.load_model('/content/drive/MyDrive/SubjectClustering_SEED_AllSessions_MyVersion/classifiers/'+str(target_sub)+'-Booster-'+str(iter+1))
        input_samples = Aligned_Target_Booster
        label_prob = Booster_model.predict(input_samples)
        Booster_predicited_label = np.argmax(np.squeeze(Booster_model.predict(input_samples)),axis=1)
        Booster_pred_labels_prob = np.squeeze(Booster_model.predict(input_samples))
        preds_matrix[0,:,:] = Booster_pred_labels_prob
        Booster_pred_labels_non_onehot = np.zeros((data_valid.shape[0]))
        for sample in range(data_valid.shape[0]):
          if Booster_predicited_label[sample]==0:
            Booster_pred_labels_non_onehot[sample]=0
          elif Booster_predicited_label[sample]==1:
            Booster_pred_labels_non_onehot[sample]=1
          elif Booster_predicited_label[sample]==2:
            Booster_pred_labels_non_onehot[sample]=-1
        Booster_accuracy = (sum(np.equal(label_valid,Booster_pred_labels_non_onehot[0:len(label_valid)])))/len(label_valid)
        selected_sources_accuracy[0] = Booster_accuracy

        if N>0:
          for classifier_num in range(N):
            classifier_index = sorted_classifiers[classifier_num]
            model = tf.keras.models.load_model('/content/drive/MyDrive/SubjectClustering_SEED_AllSessions_MyVersion/classifiers/'+str(target_sub)+'-'+ str(classifier_index+1) +'-'+str(iter+1))
            input_samples = Aligned_Target[classifier_num,:,:]
            label_prob = model.predict(input_samples)
            classifier_predicited_label = np.argmax(np.squeeze(model.predict(input_samples)),axis=1)
            classifier_pred_labels_prob = np.squeeze(model.predict(input_samples))
            preds_matrix[classifier_num+1,:,:] = classifier_pred_labels_prob

            classifier_pred_labels_non_onehot = np.zeros((data_valid.shape[0]))
            for sample in range(data_valid.shape[0]):
              if classifier_predicited_label[sample]==0:
                classifier_pred_labels_non_onehot[sample]=0
              elif classifier_predicited_label[sample]==1:
                classifier_pred_labels_non_onehot[sample]=1
              elif classifier_predicited_label[sample]==2:
                classifier_pred_labels_non_onehot[sample]=-1
            #print(label_valid)
            #print(classifier_pred_labels_non_onehot)
            classifier_accuracy = (sum(np.equal(label_valid,classifier_pred_labels_non_onehot[0:len(label_valid)])))/len(label_valid)
            selected_sources_accuracy[classifier_num+1] = classifier_accuracy
          #print("\naccuracy (on test samples) for classifiers in this cluster :")
          #print(selected_sources_accuracy)
          #print("\n")

        # calculate major prediction (soft voting)
        for sample in range(data_valid.shape[0]):
          sum_prob_0 = 0
          sum_prob_1 = 0
          sum_prob_min1 = 0
          for classifier in range(N+1):
            sum_prob_0 += preds_matrix[classifier,sample,0]
            sum_prob_1 += preds_matrix[classifier,sample,1]
            sum_prob_min1 += preds_matrix[classifier,sample,2]
          #print("for sample "+str(sample))
          #print("sum_prob_0: "+str(sum_prob_0))
          #print("sum_prob_1: "+str(sum_prob_1))
          #print("sum_prob_min1: "+str(sum_prob_min1))
          sample_label = 0
          if (sum_prob_0 > sum_prob_1 and sum_prob_0 > sum_prob_min1):
            sample_label = 0
          elif (sum_prob_1 > sum_prob_0 and sum_prob_1 > sum_prob_min1):
            sample_label = 1
          elif (sum_prob_min1 > sum_prob_0 and sum_prob_min1 > sum_prob_1):
            sample_label = -1
          prediction[sample] = sample_label
          #print("predicted label: "+str(sample_label)+"\n")

        # compute the accuracy for target test samples
        acc_iter = (sum(np.equal(label_valid,prediction[0:len(label_valid)])))/len(label_valid)
        CLUSTER_ACCURACY[N_counter] = 100*acc_iter
        N_counter += 1
        print('\naccuracy for cluster '+str(K+1)+" with N = "+str(N)+" in this iteration : "+str(acc_iter))
      if CLUSTER_ACCURACY[0]>High_acc_in_iter:
        High_acc_in_iter = CLUSTER_ACCURACY[0]
      print("\n")
      plt.title("Accuracy Vs N (Number of Selected Classifiers) [Cluster: "+str(K+1)+" of "+str(best_num_cluster)+"][iteration: "+str(iter+1)+" of 5]")
      plt.ylabel('Accuracy (%)')
      plt.xlabel('N (Number of Selected Classifiers)')
      plt.xticks(np.arange(N+1), np.arange(0, N+2))
      plt.plot(CLUSTER_ACCURACY, color="blue", marker = '*')
      plt.show()
      if K==K_Selected_MMPP:
        iter_acc_list_selected_MMPP.append(CLUSTER_ACCURACY)
      if K==K_clustering:
        iter_acc_list_clustering.append(CLUSTER_ACCURACY)
        #print("iter_acc_list")
        #print(iter_acc_list)
      if K==K_Selected_MNCC:
        iter_acc_list_selected_MNCC.append(CLUSTER_ACCURACY)
      if K==K_Selected_MMA:
        iter_acc_list_selected_MMA.append(CLUSTER_ACCURACY)
      if K==K_Selected_MTA:
        iter_acc_list_selected_MTA.append(CLUSTER_ACCURACY)
    iter_acc_list_HIGHEST.append(High_acc_in_iter)
    print("Potential Highest Acc: "+str(iter_acc_list_HIGHEST))
  Target_acc_list_selected_MMPP.append(iter_acc_list_selected_MMPP)
  Target_acc_list_selected_MNCC.append(iter_acc_list_selected_MNCC)
  Target_acc_list_selected_MMA.append(iter_acc_list_selected_MMA)
  Target_acc_list_selected_MTA.append(iter_acc_list_selected_MTA)
  Target_acc_list_clustering.append(iter_acc_list_clustering)
  Target_acc_list_HIGHEST.append(iter_acc_list_HIGHEST)
  print("Potential Highest Acc for Target subject: "+str(Target_acc_list_HIGHEST))
  #print("accuracy for this target (N=0)(MMPP): ")
  #print(Target_acc_list)
  #print("accuracy for this target (N=0)(By Clustering): ")
  #print(Target_acc_list)
  #print("\n\n********* accuracy for subject "+str(target_sub)+" as target: "+str(np.mean(ACC_MAT_iter[target_sub-1,:]))+" *********\n\n\n")

# **Results**
The following results were obtained for **subjects 1 to 5** as target subjects. These results should be saved in the corresponding results sheet.

* **Note**: Considering only the first item of *iter_acc* is equivalent to **using only booster classifier** (**N=0**). This means that the trained classifiers that were used in the cluster selection stage were not used in the final classification. In order to maintain the comprehensiveness and generalizability of the code, it has been implemented in this way.

In [ ]:
# Optimal Cluster Selection Metric: Choosing by Clustering
TargetIter_AccMat = np.zeros((15, 5))

# Iterate through each target subject
for target in range(5):
  target_acc = Target_acc_list_clustering[target]

  # Iterate through each iteration for the current target
  for iter in range(5):
    iter_acc = target_acc[iter]
    iteration_accuracy = iter_acc[0]
    TargetIter_AccMat[target, iter] = iteration_accuracy

# Calculate the mean accuracy for each target subject
Target_Acc = np.mean(TargetIter_AccMat, axis=1)
print("Mean accuracy for each target:", Target_Acc)

# Calculate the overall mean accuracy
overall_acc = np.mean(Target_Acc)
print("Overall mean accuracy:", overall_acc)

In [ ]:
# Optimal Cluster Selection Metric: Maximum Major Prediction Proportion
TargetIter_AccMat = np.zeros((15, 5))

# Iterate through each target subject
for target in range(5):
  target_acc = Target_acc_list_selected_MMPP[target]

  # Iterate through each iteration for the current target subject
  for iter in range(5):
    iter_acc = target_acc[iter]
    iteration_accuracy = iter_acc[0]
    TargetIter_AccMat[target, iter] = iteration_accuracy

# Calculate the mean accuracy for each target subject
Target_Acc = np.mean(TargetIter_AccMat, axis=1)
print("Mean accuracy for each target:", Target_Acc)

# Calculate the overall mean accuracy
overall_acc = np.mean(Target_Acc)
print("Overall mean accuracy:", overall_acc)

In [ ]:
# Optimal Cluster Selection Metric: Maximum Correctly Predicted Classifiers Proportion
TargetIter_AccMat = np.zeros((15, 5))

# Iterate through each target subject
for target in range(5):
  target_acc = Target_acc_list_selected_MNCC[target]

  # Iterate through each iteration for the current target subject
  for iter in range(5):
    iter_acc = target_acc[iter]
    iteration_accuracy = iter_acc[0]
    TargetIter_AccMat[target, iter] = iteration_accuracy

# Calculate the mean accuracy for each target subject
Target_Acc = np.mean(TargetIter_AccMat, axis=1)
print("Mean accuracy for each target:", Target_Acc)

# Calculate the overall mean accuracy
overall_acc = np.mean(Target_Acc)
print("Overall mean accuracy:", overall_acc)

In [ ]:
# Optimal Cluster Selection Metric: Maximum Mean Accuracy
TargetIter_AccMat = np.zeros((15, 5))

# Iterate through each target subject
for target in range(5):
  target_acc = Target_acc_list_selected_MMA[target]

  # Iterate through each iteration for the current target subject
  for iter in range(5):
    iter_acc = target_acc[iter]
    iteration_accuracy = iter_acc[0]
    TargetIter_AccMat[target, iter] = iteration_accuracy

# Calculate the mean accuracy for each target subject
Target_Acc = np.mean(TargetIter_AccMat, axis=1)
print("Mean accuracy for each target:", Target_Acc)

# Calculate the overall mean accuracy
overall_acc = np.mean(Target_Acc)
print("Overall mean accuracy:", overall_acc)

In [ ]:
# Optimal Cluster Selection Metric: Maximum Top Accuracy
TargetIter_AccMat = np.zeros((15, 5))

# Iterate through each target subject
for target in range(5):
  target_acc = Target_acc_list_selected_MTA[target]

  # Iterate through each iteration for the current target subject
  for iter in range(5):
    iter_acc = target_acc[iter]
    iteration_accuracy = iter_acc[0]
    TargetIter_AccMat[target, iter] = iteration_accuracy

# Calculate the mean accuracy for each target subject
Target_Acc = np.mean(TargetIter_AccMat, axis=1)
print("Mean accuracy for each target:", Target_Acc)

# Calculate the overall mean accuracy
overall_acc = np.mean(Target_Acc)
print("Overall mean accuracy:", overall_acc)

In [ ]:
# Best available accuracy: Assuming that in each iteration, the best cluster is selected in some way
# We use this accuracy, as well as the closeness of the accuracy of the abovementioned metrics, to evaluate the modification in the algorithm

TargetIter_AccMat = np.zeros((15, 5))

# Iterate through each target
for target in range(5):
  target_acc = Target_acc_list_HIGHEST[target]

  # Iterate through each iteration for the current target
  for iter in range(5):
    iter_acc = target_acc[iter]
    TargetIter_AccMat[target, iter] = iter_acc

# Calculate the mean accuracy for each target
Target_Acc = np.mean(TargetIter_AccMat, axis=1)
print("Mean accuracy for each target:", Target_Acc)

# Calculate the overall mean accuracy
overall_acc = np.mean(Target_Acc)
print("Overall mean accuracy:", overall_acc)